In [1]:
import glob
import shutil
import numpy as np
import dask as d

from numpy.random import RandomState

In [2]:
from fl_tissue_model_tools import data_prep

In [3]:
root_data = "D:/malaria_data"
unsplit_data = f"{root_data}/cell_images"
split_data = f"{root_data}/cell_images_split"

In [4]:
mal_img_names = np.array([img_n.replace("\\", "/") for img_n in glob.glob(f"{root_data}/cell_images/Parasitized/*.png")])
ctrl_img_names = np.array([img_n.replace("\\", "/") for img_n in glob.glob(f"{root_data}/cell_images/Uninfected/*.png")])

In [5]:
data_prep.make_dir(f"{root_data}/cell_images_split/train/parasitized")
data_prep.make_dir(f"{root_data}/cell_images_split/train/uninfected")
data_prep.make_dir(f"{root_data}/cell_images_split/test/parasitized")
data_prep.make_dir(f"{root_data}/cell_images_split/test/uninfected")

In [6]:
n_mal = len(mal_img_names)
n_ctrl = len(ctrl_img_names)
test_prop = 0.2

In [7]:
n_test_ctrl, n_test_mal = round(test_prop * n_ctrl), round(test_prop * n_mal)
n_train_ctrl, n_train_mal = round(n_ctrl -  n_test_ctrl), round(n_mal - n_test_mal)
# n_test = round(test_prop * n_ctrl), round(test_prop * n_mal)
# n_train = round(n_ctrl -  n_test[0]), round(n_mal - n_test[1])

In [8]:
# 0 for train, 1 for test
train_test_ids = np.array([0, 1])
mal_reps = np.array([n_train_mal, n_test_mal])
ctrl_reps = np.array([n_train_ctrl, n_test_ctrl])

In [9]:
# Generate shuffled train/test split index arrays
rs = RandomState(seed=2984)
mal_split = np.repeat(train_test_ids, mal_reps)
ctrl_split = np.repeat(train_test_ids, ctrl_reps)
rs.shuffle(mal_split)
rs.shuffle(ctrl_split)

In [10]:
a = "apples.txt"
a.split("/")[-1]

'apples.txt'

In [11]:
ctrl_split

array([0, 0, 0, ..., 0, 1, 0])

In [12]:
split_map = {0: "train", 1: "test"}
# save_class_imgs(ctrl_img_names, ctrl_split, split_map, "Uninfected", f"{root_data}/cell_images_split")

d.compute(
    d.delayed(data_prep.save_class_imgs)(ctrl_img_names, ctrl_split, split_map, "uninfected", f"{root_data}/cell_images_split"),
    d.delayed(data_prep.save_class_imgs)(mal_img_names, mal_split, split_map, "parasitized", f"{root_data}/cell_images_split")
)

(None, None)